In [ ]:
# %pip install python-dotenv
# %pip install pandas
# %pip install geopandas
# %pip install seaborn

In [ ]:
import os
import dotenv
import geopandas as gpd
import pandas as pd

dotenv.load_dotenv()

geopackage_name = 'output/wind-turbines-capacity.gpkg'

CSV data prepared from the U.S. Office of Energy Efficiency and Renewable Energy's WINDExchange, https://windexchange.energy.gov/maps-data/321

In [ ]:
installed_capacity_df = pd.read_csv(os.path.join(os.environ['DATA_DIR'], 'WindTurbines/capacity/installed-capacity.csv'))
installed_capacity_df['installed_capacity'] = installed_capacity_df['installed_capacity'].fillna(0)
installed_capacity_df.head()

In [ ]:
potential_capacity_df = pd.read_csv(os.path.join(os.environ['DATA_DIR'], 'WindTurbines/capacity/potential-capacity.csv'))
potential_capacity_df.head()

In [ ]:
countries_50m = gpd.read_file(os.path.join(os.environ['NATURAL_EARTH_DIR'], 'Countries/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp'))
states_provinces_50m = gpd.read_file(os.path.join(os.environ['NATURAL_EARTH_DIR'], 'StatesProvinces/ne_50m_admin_1_states_provinces_lakes/ne_50m_admin_1_states_provinces_lakes.shp'))

In [ ]:
na_countries = countries_50m[countries_50m['CONTINENT'] == 'North America']
us_states_50m = states_provinces_50m[states_provinces_50m['iso_a2'] == 'US']

In [ ]:
us_states_50m = us_states_50m[["iso_3166_2", "name", "geometry"]]
us_states_50m["state_abr"] = us_states_50m["iso_3166_2"].str.split("-", expand=True)[1]
us_states_50m = us_states_50m.merge(potential_capacity_df[['state_abr', 'potential_capacity']], on='state_abr', how='left')
us_states_50m = us_states_50m.merge(installed_capacity_df[['name', 'installed_capacity']], on='name', how='left')
us_states_50m['percent_installed'] = (us_states_50m['installed_capacity'] / us_states_50m['potential_capacity']) * 100
us_states_50m = us_states_50m.dropna()

us_states_50m.head()

In [ ]:
na_countries.to_file(geopackage_name, layer='na_countries_50m', driver='GPKG')
us_states_50m.to_file(geopackage_name, layer='us_states_50m', driver='GPKG')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/Windows/Fonts/GILSANUB.TTF'
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))

ax = sns.barplot(data=us_states_50m[["name", "percent_installed"]], x="name", y="percent_installed", palette="Set3")

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=9)

for patch in ax.patches:
    patch.set_width(0.8)

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

ax.set_xlabel("State")
ax.set_ylabel("Percent of potential capacity installed")

plt.show()